In [ ]:
%matplotlib inline

# **PyTorch Quickstart**

이번 강좌에서는 신경망의 일반적인 작업들을 위한 PyTorch API를 살펴본다.  이 강좌는 [PyTorch 공식 튜토리얼](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)의 [한글 번역본](https://tutorials.pytorch.kr/beginner/basics/quickstart_tutorial.html)을 다시 부분적으로 수정한 것이다.

## 데이터 다루기

파이토치(PyTorch)는 **데이터 작업을 위한 2가지 기본 요소**인
``torch.utils.data.DataLoader`` 와 ``torch.utils.data.Dataset``를 제공한다.
``Dataset``은 데이터와 정답(label)을 저장하고, ``DataLoader``는 ``Dataset``을 순회 가능한 객체(iterable)로 만들어준다.


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch는 [`TorchText`](https://pytorch.org/text/stable/index.html), [`TorchVision`](https://pytorch.org/vision/stable/index.html) 및
[`TorchAudio`](https://pytorch.org/audio/stable/index.html)와 같이 응용분야에 특화된 라이브러리를 다양한 데이터셋과 함께 제공한다.
이 강좌에서는 `TorchVision`이 제공하는 데이터셋 중에에 하나를 사용한다.

``torchvision.datasets`` 모듈은 CIFAR, COCO 등과 같은 다양한 실제 비전(vision) 데이터에 대한 ``Dataset`` (전체 목록은 [여기](https://pytorch.org/vision/stable/datasets.html)를 참조)을 포함하고 있다.
이 튜토리얼에서는 그 중 `FasionMNIST` 데이터셋을 사용한다.

모든 TorchVision ``Dataset``은 매개변수 ``transform``과 ``target_transform``을 받아들이는데 이를 통해 원시(raw) 데이터와 정답(label)을 적절하게 전처리(preprocessing)하기 위한 절차를 지정할 수 있다. 여기에서는 전처리 절차로 `torchvision.transforms`이 제공하는 `ToTensor`를 적용하였다. `ToTensor`는 이미지의 픽셀값을 `0~1` 사이의 실수로 정규화하고, `Tensor` 데이터 타입으로 변환하는 일을 한다.


In [ ]:
# 공개 데이터셋에서 학습 데이터를 내려받습니다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:03<00:00, 8314301.31it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 142949.22it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 2667470.14it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 19773147.43it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



``DataLoader``는 데이터셋을 순회 가능한 객체(iterable)로 감싸고(wrap), 자동화된 배치(batch), 샘플링(sampling),
섞기(shuffle) 및 다중 프로세스로 데이터 불러오기(multiprocess data loading)를 지원한다. 여기서는 배치 크기(batch size)를 64로 정의한다.
즉, 데이터로더(dataloader) 객체의 각 요소는 64개의 특징(feature)과 정답(label)을 묶음(batch)으로 반환한다.



In [ ]:
batch_size = 64

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: {}".format(X.shape))
    print("Shape of y: {} {}".format(y.shape, y.dtype))
    print(y)
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
tensor([9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 7, 3, 4, 1, 2, 4, 8, 0, 2, 5, 7, 9,
        1, 4, 6, 0, 9, 3, 8, 8, 3, 3, 8, 0, 7, 5, 7, 9, 6, 1, 3, 7, 6, 7, 2, 1,
        2, 2, 4, 4, 5, 8, 2, 2, 8, 4, 8, 0, 7, 7, 8, 5])


**Note:** 하나의 이미지의 `shape`이 `[1, 28, 28]`인 점에 주의하라. 보통의 Python 이미지 라이브러리에서는 이미지가 `[height, width, num_channels]`의 형태로 표현되는 것과 달리 PyTorch에서는 `[num_channels, height, width]`의 형태로 표현된다는 것을 보여준다.

------------------------------------------------------------------------------------------

## 모델 만들기

PyTorch에서 신경망 모델은 일반적으로 [`nn.Module`](<https://pytorch.org/docs/stable/generated/torch.nn.Module.html>)을
상속받는 클래스(class)를 정의하여 생성한다. ``__init__`` 함수에서 신경망의 계층(layer)들을 정의하고, ``forward`` 함수에서
신경망에 데이터를 어떻게 전달할지 지정한다. 가능한 경우 GPU로 신경망을 이동시켜 연산을 가속(accelerate)한다.



In [ ]:
# 학습에 사용할 CPU나 GPU 장치를 얻습니다.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# 모델을 정의합니다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        logits = self.relu(self.fc2(x))
        return logits

model = NeuralNetwork()

model = model.to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
  (relu): ReLU()
)


In [ ]:
print(model.parameters())
for name, param in model.named_parameters():
    # if param.requires_grad:
    print(name, param.data.shape)


<generator object Module.parameters at 0x7fea4bcc0270>
fc1.weight torch.Size([512, 784])
fc1.bias torch.Size([512])
fc2.weight torch.Size([10, 512])
fc2.bias torch.Size([10])


------------------------------------------------------------------------------------------




모델 매개변수 최적화하기
------------------------------------------------------------------------------------------
모델을 학습하려면 [손실 함수(loss function)](<https://pytorch.org/docs/stable/nn.html#loss-functions>)와
[옵티마이저(optimizer)](<https://pytorch.org/docs/stable/optim.html>)가 필요하다.



In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

각 학습 단계(training loop)에서 모델은 배치(batch)로 제공되는 학습 데이터셋에 대한 예측을 수행하고,
예측 오류를 역전파(backpropagate)하여 모델의 매개변수를 조정한다.



In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        if batch == 0:
          print(pred)
          print(pred.shape)
          print(pred.dtype)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

모델이 학습하고 있는지를 확인하기 위해 테스트 데이터셋으로 모델의 성능을 확인해보는 함수를 작성한다.



In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

학습은 여러번의 반복 단계(epochs)를 거쳐서 수행된다. 각 에폭마다 모델의 정확도(accuracy)와 손실(loss)을 출력한다; 에폭마다 정확도가 증가하고 손실이 감소하는 것을 보려고 한다.



In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
tensor([[0.0000e+00, 7.0141e-02, 5.8377e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 2.0183e-01, 3.0878e-02, 3.2531e-02],
        [0.0000e+00, 1.6288e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 1.8032e-01, 0.0000e+00, 0.0000e+00],
        [4.1099e-02, 3.4616e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 1.4095e-02, 0.0000e+00, 0.0000e+00],
        [9.8872e-02, 7.1755e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 2.6174e-02, 5.8057e-02, 1.9072e-02],
        [1.2525e-01, 5.9915e-02, 1.3943e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 1.0445e-01, 4.6586e-02, 0.0000e+00],
        [9.6771e-02, 4.1863e-02, 9.7055e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 1.0262e-01, 9.9090e-02, 1.2296e-02],
        [0.0000e+00, 0.0000e+00, 7.3466e-02, 0.0000e+00, 0.0000e+00, 6.1315e-03,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 

------------------------------------------------------------------------------------------




모델 저장하기
------------------------------------------------------------------------------------------
모델을 저장하는 일반적인 방법은 (모델의 매개변수들을 포함하여) 내부 상태 사전(internal state dictionary)을
직렬화(serialize)하는 것이다.



In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


모델 불러오기
------------------------------------------------------------------------------------------

모델을 불러오는 과정에는 모델 구조를 다시 만들고 상태 사전을 모델에 불러오는 과정이 포함된다.



In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

이제 이 모델을 사용해서 예측을 할 수 있다.



In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
print(x.shape)
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

torch.Size([1, 28, 28])
Predicted: "Ankle boot", Actual: "Ankle boot"
